## 参考资料

中国科学院软件研究所 刘焕勇老师 https://github.com/liuhuanyong/QASystemOnMedicalKG

## 导入工具包

In [1]:
import os
import json
from py2neo import Graph, Node

import pandas as pd

## 导入数据集csv文件

In [2]:
df = pd.read_csv('medical_data.csv')

In [3]:
df.shape

(21, 23)

## 实体（节点）

### 实体：所有症状

In [4]:
symptoms = []
for each in df['症状']:
    symptoms.extend(each.split(','))
symptoms = set(symptoms)

### 所有科室

In [5]:
departments = []
for each in df['科室']:
    departments.extend(each.split(','))
departments = set(departments)

### 实体：所有检查

In [6]:
checks = []
for each in df['检查']:
    checks.extend(each.split(','))
checks = set(checks)

### 实体：所有药物

In [7]:
drugs = []
for each in df['推荐药物']:
    try:
        drugs.extend(each.split(','))
    except:
        pass
for each in df['常用药物']:
    try:
        drugs.extend(each.split(','))
    except:
        pass
drugs = set(drugs)

### 实体：所有食物

In [8]:
foods = []
for each in df['可以吃']:
    try:
        foods.extend(each.split(','))
    except:
        pass
for each in df['不可以吃']:
    try:
        foods.extend(each.split(','))
    except:
        pass
for each in df['推荐吃']:
    try:
        foods.extend(each.split(','))
    except:
        pass
foods = set(foods)

### 实体：所有药物厂商

In [9]:
producers = []

for each in df['具体药物']:
    try:
        for each_drug in each.split(','):
            producer = each_drug.split('(')[0]
            producers.append(producer)
    except:
        pass
producers = set(producers)

### 疾病字典信息

In [10]:
disease_infos = [] # 疾病信息
for idx, row in df.iterrows():
    disease_infos.append(dict(row))

In [11]:
dict(row).keys()

dict_keys(['疾病名称', '疾病描述', '疾病种类', '科室', '病因', '症状', '检查', '并发症', '花费', '疗程', '疗法', '治愈率', '易感人群', '感染概率', '感染途径', '预防措施', '推荐药物', '常用药物', '具体药物', '可以吃', '不可以吃', '推荐吃', '是否纳入医保'])

## 关系（边）

In [12]:
def deduplicate(rels_old):
    '''关系去重函数'''
    rels_new = []
    for each in rels_old:
        if each not in rels_new:
            rels_new.append(each)
    return rels_new

### 关系：疾病-检查

In [13]:
rels_check = []
for idx, row in df.iterrows():
    for each in row['检查'].split(','):
        rels_check.append([row['疾病名称'], each])
rels_check = deduplicate(rels_check)

In [14]:
rels_check

[['肺泡蛋白质沉积症', '胸部CT检查'],
 ['肺泡蛋白质沉积症', '肺活检'],
 ['肺泡蛋白质沉积症', '支气管镜检查'],
 ['百日咳', '耳、鼻、咽拭子细菌培养'],
 ['百日咳', '周围血白细胞计数及分类检验'],
 ['百日咳', '血常规'],
 ['百日咳', '酶联免疫吸附试验'],
 ['百日咳', '白细胞分类计数'],
 ['苯中毒', '血常规'],
 ['苯中毒', '骨髓象分析'],
 ['苯中毒', '先令氏指数'],
 ['喘息样支气管炎', '肺部检查'],
 ['喘息样支气管炎', '肺和胸膜听诊'],
 ['喘息样支气管炎', '抗链球菌型M蛋白抗体'],
 ['喘息样支气管炎', '抗链球菌壁多糖抗体'],
 ['喘息样支气管炎', '酶联免疫吸附试验'],
 ['成人呼吸窘迫综合征', '胸部CT检查'],
 ['成人呼吸窘迫综合征', '呼吸肌功能测定'],
 ['成人呼吸窘迫综合征', '血浆蛋白C抗原'],
 ['成人呼吸窘迫综合征', '肺泡气-动脉血氧分压差'],
 ['成人呼吸窘迫综合征', '肺毛细血管楔压'],
 ['大量羊水吸入', '肺部检查'],
 ['大量羊水吸入', '胸部透视'],
 ['大量羊水吸入', '胸部平片'],
 ['大量羊水吸入', '胸部CT检查'],
 ['单纯性肺嗜酸粒细胞浸润症', '痰液中细胞分类'],
 ['单纯性肺嗜酸粒细胞浸润症', '胸部平片'],
 ['单纯性肺嗜酸粒细胞浸润症', '痰液中寄生虫和虫卵'],
 ['大叶性肺炎', 'Optochin敏感试验'],
 ['大叶性肺炎', '小白鼠毒力试验'],
 ['大叶性肺炎', '痰培养'],
 ['大叶性肺炎', '肺活量体重指数'],
 ['大叶性肺炎', '胸部平片'],
 ['大叶性肺炎', '免疫电泳'],
 ['大叶性肺炎', '血常规'],
 ['大叶性肺炎', '痰液细菌涂片检查'],
 ['大楼病综合征', '钼靶X线检查'],
 ['大楼病综合征', 'CT检查'],
 ['大楼病综合征', '血常规'],
 ['二硫化碳中毒', '血常规'],
 ['二硫化碳中毒', '尿常规'],
 ['二硫化碳中毒', '肾功能检查'],
 ['二硫化碳中毒', '神经系统检查

### 关系：疾病-症状

In [15]:
rels_symptom = []
for idx, row in df.iterrows():
    for each in row['症状'].split(','):
        rels_symptom.append([row['疾病名称'], each])
rels_symptom = deduplicate(rels_symptom)

In [16]:
rels_symptom

[['肺泡蛋白质沉积症', '紫绀'],
 ['肺泡蛋白质沉积症', '胸痛'],
 ['肺泡蛋白质沉积症', '呼吸困难'],
 ['肺泡蛋白质沉积症', '乏力'],
 ['肺泡蛋白质沉积症', '毓卓'],
 ['百日咳', '吸气时有蝉鸣音'],
 ['百日咳', '痉挛性咳嗽'],
 ['百日咳', '胸闷'],
 ['百日咳', '肺阴虚'],
 ['百日咳', '抽搐'],
 ['百日咳', '低热'],
 ['百日咳', '闫鹏辉'],
 ['百日咳', '惊厥'],
 ['苯中毒', '恶心'],
 ['苯中毒', '抽搐'],
 ['苯中毒', '感觉障碍'],
 ['喘息样支气管炎', '耸肩喘息'],
 ['喘息样支气管炎', '哮鸣音'],
 ['喘息样支气管炎', '纤毛上皮细胞损伤脱落'],
 ['喘息样支气管炎', '变应性咳嗽'],
 ['喘息样支气管炎', '化学性支气管炎'],
 ['喘息样支气管炎', '喘息'],
 ['喘息样支气管炎', '冬春季的慢性咳...'],
 ['喘息样支气管炎', '咳嗽伴哮鸣音'],
 ['成人呼吸窘迫综合征', '呼吸困难'],
 ['成人呼吸窘迫综合征', '紫绀'],
 ['成人呼吸窘迫综合征', '心源性呼吸窘迫'],
 ['大量羊水吸入', '面色青紫'],
 ['大量羊水吸入', '呼吸困难'],
 ['大量羊水吸入', '口唇青紫'],
 ['大量羊水吸入', '肺纹理增粗'],
 ['大量羊水吸入', '出生后即有持续青紫'],
 ['大量羊水吸入', '口唇和甲床略带青紫'],
 ['大量羊水吸入', '气急'],
 ['单纯性肺嗜酸粒细胞浸润症', '嗜酸性粒细胞增多'],
 ['单纯性肺嗜酸粒细胞浸润症', '咽部异物感'],
 ['单纯性肺嗜酸粒细胞浸润症', '胸闷憋气'],
 ['单纯性肺嗜酸粒细胞浸润症', '咯血伴发热'],
 ['单纯性肺嗜酸粒细胞浸润症', '胸闷'],
 ['单纯性肺嗜酸粒细胞浸润症', '乏力'],
 ['大叶性肺炎', '湿啰音'],
 ['大叶性肺炎', '胸痛'],
 ['大叶性肺炎', '发烧'],
 ['大叶性肺炎', '咳铁锈色痰'],
 ['大叶性肺炎', '急性面容'],
 ['大叶性肺炎', '呼吸音减弱'],
 ['大

### 关系：疾病-疾病（并发症）

In [17]:
rels_acompany = []
for idx, row in df.iterrows():
    for each in row['并发症'].split(','):
        rels_acompany.append([row['疾病名称'], each])
rels_acompany = deduplicate(rels_acompany)

In [18]:
rels_acompany

[['肺泡蛋白质沉积症', '多重肺部感染'],
 ['百日咳', '肺不张'],
 ['苯中毒', '贫血'],
 ['喘息样支气管炎', '支气管哮喘'],
 ['成人呼吸窘迫综合征', '细菌性肺炎'],
 ['大量羊水吸入', '呼吸衰竭'],
 ['单纯性肺嗜酸粒细胞浸润症', '胆道蛔虫病'],
 ['大叶性肺炎', '脓胸'],
 ['大楼病综合征', '抑郁症'],
 ['二硫化碳中毒', '昏迷'],
 ['肺-胸膜阿米巴病', '阿米巴肝脓肿'],
 ['肺出血－肾炎综合征', '便血'],
 ['肺放线菌病', '膈下脓肿'],
 ['肺泡蛋白沉着症', '呼吸衰竭'],
 ['肺曲菌病', '过敏性鼻炎'],
 ['放射性肺炎', '肺气肿'],
 ['肺念珠菌病', '菌血症'],
 ['肺大疱', '张力性气胸'],
 ['肺炎球菌肺炎', '败血症'],
 ['肺气肿', '呼吸衰竭'],
 ['肺气肿', '自发性气胸'],
 ['肺气肿', '慢性肺源性心脏病'],
 ['肺气肿', '胃溃疡'],
 ['肺炎杆菌肺炎', '脑疝']]

### 关系：疾病-推荐药物

In [19]:
rels_recommanddrug = []
for idx, row in df.iterrows():
    try:
        for each in row['推荐药物'].split(','):
            rels_recommanddrug.append([row['疾病名称'], each])
    except:
        pass
rels_recommanddrug = deduplicate(rels_recommanddrug)

### 关系：疾病-常用药物

In [20]:
rels_commonddrug = []
for idx, row in df.iterrows():
    try:
        for each in row['常用药物'].split(','):
            rels_commonddrug.append([row['疾病名称'], each])
    except:
        pass
rels_commonddrug = deduplicate(rels_commonddrug)

### 关系：疾病-不可以吃

In [21]:
rels_noteat = []
for idx, row in df.iterrows():
    try:
        for each in row['不可以吃'].split(','):
            rels_noteat.append([row['疾病名称'], each])
    except:
        pass
rels_noteat = deduplicate(rels_noteat)

### 关系：疾病-可以吃

In [22]:
rels_doeat = []
for idx, row in df.iterrows():
    try:
        for each in row['可以吃'].split(','):
            rels_doeat.append([row['疾病名称'], each])
    except:
        pass
rels_doeat = deduplicate(rels_doeat)

### 关系：疾病-推荐吃

In [23]:
rels_recommandeat = []
for idx, row in df.iterrows():
    try:
        for each in row['推荐吃'].split(','):
            rels_recommandeat.append([row['疾病名称'], each])
    except:
        pass
rels_recommandeat = deduplicate(rels_recommandeat)

### 关系：药物厂商-具体药物

In [24]:
rels_drug_producer = []
for each in df['具体药物']:
    try:
        for each_drug in each.split(','):
            producer = each_drug.split('(')[0]
            drug = each_drug.split('(')[1][:-1]
            rels_drug_producer.append([producer, drug])
    except:
        pass
rels_drug_producer = deduplicate(rels_drug_producer)

### 关系：疾病-科室、小科室-大科室

In [25]:
rels_category = [] # 关系：疾病-科室
rels_department = [] # 关系：小科室-大科室
for idx, row in df.iterrows():
    if len(row['科室'].split(',')) == 1:
        rels_category.append([row['疾病名称'], row['科室']])
    else:
        big = row['科室'].split(',')[0] # 大科室
        small = row['科室'].split(',')[1] # 小科室
        rels_category.append([row['疾病名称'], small])
        rels_department.append([small, big])
rels_category = deduplicate(rels_category)
rels_department = deduplicate(rels_department)

In [26]:
rels_category

[['肺泡蛋白质沉积症', '呼吸内科'],
 ['百日咳', '小儿内科'],
 ['苯中毒', '急诊科'],
 ['喘息样支气管炎', '呼吸内科'],
 ['成人呼吸窘迫综合征', '呼吸内科'],
 ['大量羊水吸入', '小儿内科'],
 ['单纯性肺嗜酸粒细胞浸润症', '呼吸内科'],
 ['大叶性肺炎', '呼吸内科'],
 ['大楼病综合征', '其他综合'],
 ['二硫化碳中毒', '急诊科'],
 ['肺-胸膜阿米巴病', '呼吸内科'],
 ['肺出血－肾炎综合征', '呼吸内科'],
 ['肺放线菌病', '呼吸内科'],
 ['肺泡蛋白沉着症', '呼吸内科'],
 ['肺曲菌病', '呼吸内科'],
 ['放射性肺炎', '呼吸内科'],
 ['肺念珠菌病', '呼吸内科'],
 ['肺大疱', '呼吸内科'],
 ['肺炎球菌肺炎', '呼吸内科'],
 ['肺气肿', '呼吸内科'],
 ['肺炎杆菌肺炎', '呼吸内科']]

In [27]:
rels_department

[['呼吸内科', '内科'], ['小儿内科', '儿科'], ['其他综合', '其他科室']]

## 连接图数据库

In [28]:
# 注意，这里的用户名为neo4j全局用户名，而非DBMS或者database的名称
g = Graph('http://localhost:7474', auth=('neo4j', 'ABC123'))

## 创建知识图谱实体（节点）

### 样例代码

In [29]:
# # Neo4j样例代码

# 创建实体
# node = Node('Disease', name='百日咳', easy_get='多见于小儿')
# g.create(node)

In [30]:
# 删除所有实体和关系
cypher = 'MATCH (n) DETACH DELETE n'
g.run(cypher)

(No data)

### 创建疾病实体

In [31]:
count = 0
for disease_dict in disease_infos:
    try:
        node = Node("Disease",
                    name=disease_dict['疾病名称'],
                    desc=disease_dict['疾病描述'],
                    prevent=disease_dict['预防措施'],
                    cause=disease_dict['病因'],
                    easy_get=disease_dict['易感人群'],
                    cure_lasttime=disease_dict['疗程'],
                    cure_department=disease_dict['科室'],
                    cure_way=disease_dict['疗法'], 
                    cured_prob=disease_dict['治愈率'])
        g.create(node)
        count += 1
        print('创建疾病实体：', disease_dict['疾病名称'])
    except:
        pass
print('共创建 {} 个疾病实体'.format(count))

创建疾病实体： 百日咳
创建疾病实体： 苯中毒
创建疾病实体： 喘息样支气管炎
创建疾病实体： 成人呼吸窘迫综合征
创建疾病实体： 大量羊水吸入
创建疾病实体： 单纯性肺嗜酸粒细胞浸润症
创建疾病实体： 大叶性肺炎
创建疾病实体： 大楼病综合征
创建疾病实体： 二硫化碳中毒
创建疾病实体： 肺-胸膜阿米巴病
创建疾病实体： 肺出血－肾炎综合征
创建疾病实体： 肺放线菌病
创建疾病实体： 肺泡蛋白沉着症
创建疾病实体： 肺曲菌病
创建疾病实体： 放射性肺炎
创建疾病实体： 肺念珠菌病
创建疾病实体： 肺大疱
创建疾病实体： 肺炎球菌肺炎
创建疾病实体： 肺气肿
创建疾病实体： 肺炎杆菌肺炎
共创建 20 个疾病实体


### 创建药物实体

In [32]:
for each in drugs:
    node = Node('Drug', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 噻托溴铵粉雾剂
创建实体 百咳静糖浆
创建实体 阿奇霉素分散片
创建实体 盐酸克林霉素棕榈酸酯颗粒
创建实体 地塞米松磷酸钠注射液
创建实体 氯霉素片
创建实体 噻托溴铵粉吸入剂
创建实体 盐酸妥洛特罗片
创建实体 小青龙合剂
创建实体 阿奇霉素胶囊
创建实体 盐酸克林霉素胶囊
创建实体 小青龙颗粒
创建实体 红霉素肠溶片
创建实体 阿莫西林颗粒
创建实体 盐酸特比萘芬溶液
创建实体 注射用盐酸溴己新
创建实体 头孢拉定分散片
创建实体 胸腺肽肠溶片
创建实体 注射用美洛西林钠舒巴坦钠
创建实体 仁青芒觉
创建实体 甲硝唑维B6片
创建实体 茶碱缓释胶囊
创建实体 硫酸沙丁胺醇气雾剂
创建实体 蒺藜药酒
创建实体 诺氟沙星胶囊
创建实体 司帕沙星分散片
创建实体 如意定喘丸
创建实体 二羟丙茶碱片
创建实体 氨苄西林胶囊
创建实体 注射用呋塞米
创建实体 酮康唑乳膏
创建实体 甲硝唑片
创建实体 喘嗽宁片
创建实体 枸橼酸喷托维林片
创建实体 氨溴特罗口服溶液
创建实体 伊曲康唑胶囊
创建实体 乳酸左氧氟沙星片
创建实体 注射用硫代硫酸钠
创建实体 克霉唑阴道片
创建实体 硫酸沙丁胺醇片
创建实体 穿心莲内酯片
创建实体 乙酰螺旋霉素片
创建实体 硝酸益康唑乳膏
创建实体 硫酸特布他林片
创建实体 头孢丙烯分散片
创建实体 喷托维林氯化铵糖浆
创建实体 硝酸咪康唑栓
创建实体 头孢克肟颗粒
创建实体 注射用盐酸氨溴索
创建实体 布地奈德气雾剂
创建实体 替硝唑片
创建实体 联苯苄唑乳膏
创建实体 头孢氨苄胶囊
创建实体 硝酸咪康唑乳膏
创建实体 大月晶丸
创建实体 化痰平喘片
创建实体 头孢克肟分散片
创建实体 环吡酮胺乳膏
创建实体 阿奇霉素颗粒
创建实体 肺气肿片
创建实体 氨茶碱片
创建实体 环酯红霉素片
创建实体 氟康唑胶囊
创建实体 阿奇霉素片
创建实体 茶碱缓释片
创建实体 盐酸利多卡因注射液
创建实体 注射用甲泼尼龙琥珀酸钠
创建实体 硝酸益康唑溶液
创建实体 托西酸舒他西林胶囊
创建实体 通宣理肺丸
创建实体 布美他尼片
创建实体 硫酸特布他林雾化液
创建实体 注射用布美他尼
创建实体 盐酸左氧氟沙星胶囊
创建实体 白及颗粒
创建实体 琥乙红霉素片
创建实体 盐酸班布特罗胶囊
创建实体 奥硝唑片
创建实体 复方蛤青胶囊
创建实体 盐酸班布特

### 创建食物实体

In [33]:
for each in foods:
    node = Node('Food', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 圆白菜
创建实体 鲤鱼
创建实体 薏米莲子粥
创建实体 海蚌
创建实体 南瓜
创建实体 鸡蛋清
创建实体 糖醋草鱼
创建实体 鹿肉
创建实体 首乌百合粥
创建实体 山楂桑椹粥
创建实体 罗汉果雪耳鸡汤
创建实体 黄瓜拌豆腐丝
创建实体 酱豆腐汁烧猪肉
创建实体 羊肉(瘦)
创建实体 海虾
创建实体 猪肉(瘦)
创建实体 番茄牛肉片
创建实体 螃蟹
创建实体 生姜乌梅饮
创建实体 韭菜
创建实体 小麦粥
创建实体 银耳粥
创建实体 炖白鲢鱼
创建实体 意式番茄牛肉凉面
创建实体 素炒小白菜
创建实体 山楂糕
创建实体 番茄皮蛋汤
创建实体 薏仁党参粥
创建实体 蔬菜杂烩粥
创建实体 山楂百合汤
创建实体 海蟹
创建实体 大蒜
创建实体 鸡蛋木耳粥
创建实体 瓜粒杂锦汤
创建实体 樱桃番茄
创建实体 海螺
创建实体 萝卜豆腐汤
创建实体 赤贝
创建实体 番茄牛肉
创建实体 核桃
创建实体 党参粥
创建实体 山药
创建实体 栗子（熟）
创建实体 莲子
创建实体 鲫鱼
创建实体 香菜
创建实体 黄芪粥
创建实体 大豆
创建实体 豆薯小米粥
创建实体 排骨汤
创建实体 空心菜
创建实体 猪肉(肥瘦)
创建实体 番茄羊肉蛋花汤
创建实体 鱼腥草
创建实体 狗肉
创建实体 鸡肝
创建实体 鲢鱼
创建实体 珊瑚白菜墩
创建实体 番茄三素
创建实体 番茄西米粥
创建实体 雪梨饮
创建实体 泡莲藕
创建实体 百合花生汤
创建实体 豆浆
创建实体 绿豆南瓜汤
创建实体 乌梅粥
创建实体 生菜
创建实体 番茄羊肉汤
创建实体 奶油
创建实体 鸡
创建实体 牛奶玉米汤
创建实体 番茄苹果芝麻蜜
创建实体 小黄瓜西瓜汁
创建实体 清蒸鸡蛋羹
创建实体 鸭梨粥
创建实体 白菜蛋花粥
创建实体 凉拌莲藕
创建实体 玉竹白菜
创建实体 凉拌番茄
创建实体 山楂红糖粥
创建实体 鲈鱼
创建实体 肉松
创建实体 茭白
创建实体 山楂乌梅饮
创建实体 小白菜
创建实体 川贝酿梨
创建实体 百合炒芦笋
创建实体 拌肚丝白菜
创建实体 黄瓜拌兔丝
创建实体 油麦菜
创建实体 番茄芹菜汁
创建实体 豆腐干炒韭菜
创建实体 椰子银耳煲鸽汤
创建实体 白酒
创建实体 南瓜子仁
创建实体 啤酒
创建实体 锅塌番茄
创建实体 松子仁
创建实体 冬菇烧白菜
创建实体 紫苏粥
创建实体 肉末冬瓜
创建实体 羊腩萝

### 创建检查实体

In [34]:
for each in checks:
    node = Node('Check', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 血浆蛋白C抗原
创建实体 肺功能检查
创建实体 肺量计检测
创建实体 免疫电泳
创建实体 Optochin敏感试验
创建实体 通气与血流灌注比值（V/Q）
创建实体 CT检查
创建实体 纤维支气管镜检查
创建实体 支气管镜检查
创建实体 痰液常规检查
创建实体 胸部透视
创建实体 胸部平片
创建实体 钼靶X线检查
创建实体 抗肾小球基底膜抗体测定（AGBM）
创建实体 尿液镜检法
创建实体 支气管造影
创建实体 心电图
创建实体 尿常规
创建实体 肺和胸膜听诊
创建实体 肺毛细血管楔压
创建实体 肺活量体重指数
创建实体 痰液细菌培养
创建实体 无效腔气量／潮气量比值
创建实体 深吸气量（IC）
创建实体 白细胞分类计数
创建实体 肺活检
创建实体 耳、鼻、咽拭子细菌培养
创建实体 肾功能检查
创建实体 肌电图
创建实体 血常规
创建实体 一秒用力呼出量／用力肺活量比值
创建实体 痰液中寄生虫和虫卵
创建实体 周围血白细胞计数及分类检验
创建实体 抗链球菌型M蛋白抗体
创建实体 肺部检查
创建实体 骨髓象分析
创建实体 脑电图检查
创建实体 痰液病原体检查
创建实体 隐血试验与含铁血黄素检查
创建实体 每分钟最大通气量（MVV）
创建实体 神经系统检查
创建实体 酶联免疫吸附试验
创建实体 眼底荧光血管造影
创建实体 肺泡气-动脉血氧分压差
创建实体 胸部CT检查
创建实体 呼吸肌功能测定
创建实体 残气量／肺总量比值（RV/TLC）
创建实体 痰培养
创建实体 小白鼠毒力试验
创建实体 痰液细菌涂片检查
创建实体 心肺功能运动试验（CPET）
创建实体 抗链球菌壁多糖抗体
创建实体 肺容量测定
创建实体 最大呼气流量-容积曲线（MEFV）
创建实体 痰液中细胞分类
创建实体 先令氏指数
创建实体 肺活量（VC）


### 创建科室实体

In [35]:
for each in departments:
    node = Node('Department', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 内科
创建实体 小儿内科
创建实体 其他科室
创建实体 呼吸内科
创建实体 急诊科
创建实体 儿科
创建实体 其他综合


### 创建 药物厂商 实体

In [36]:
for each in producers:
    node = Node('Producer', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 艾富西药业盐酸氨溴索葡萄糖
创建实体 金诃藏药大月晶丸
创建实体 山西云鹏制药氨茶碱片
创建实体 佐今明百咳静糖浆
创建实体 一品红盐酸克林霉素棕榈酸酯
创建实体 美扶
创建实体 罗欣药业注射用盐酸氨溴索
创建实体 东信药业蛇胆川贝液
创建实体 央宗药业蒺藜药酒
创建实体 广东环球制药茶碱缓释片
创建实体 宜昌人福盐酸克林霉素胶囊
创建实体 盐酸克林霉素胶囊
创建实体 金诃藏药仁青芒觉
创建实体 蜀阳药业人血白蛋白
创建实体 桂林华信环吡酮胺乳膏
创建实体 苏州三药盐酸克林霉素胶囊
创建实体 平光制药红霉素肠溶片
创建实体 天津太平洋二羟丙茶碱片
创建实体 仁和堂硫酸沙丁胺醇片
创建实体 鲁抗麦迪霉素片
创建实体 太原药业甲硝唑片
创建实体 全星制药注射用盐酸头孢替安
创建实体 瑞阳制药注射用磺苄西林钠
创建实体 齐迈星
创建实体 白敬宇制药氨茶碱片
创建实体 博康健制药盐酸氨溴索葡萄糖
创建实体 欣匹特
创建实体 锦华药业盐酸克林霉素胶囊
创建实体 恒安药业盐酸特比萘芬片
创建实体 润阳药业盐酸氨溴索葡萄糖注
创建实体 海南制药厂蛇胆川贝液
创建实体 百利药业阿奇霉素颗粒
创建实体 朗瑞
创建实体 尤金
创建实体 西藏甘露仁青芒觉
创建实体 开林
创建实体 衡阳紫光古汉硝酸益康唑乳膏
创建实体 康美药业红霉素肠溶片
创建实体 双林生物人血白蛋白
创建实体 潇然
创建实体 澳利达奈德注射用盐酸溴己新
创建实体 安必君氨苄西林胶囊
创建实体 江苏长江阿奇霉素胶囊
创建实体 上海新亚头孢克肟分散片
创建实体 永定制药百咳静糖浆
创建实体 全星制药注射用布美他尼
创建实体 遂成药业地塞米松磷酸钠注射
创建实体 辽宁海思科注射用甲泼尼龙琥
创建实体 哈药总厂头孢氨苄甲氧苄啶片
创建实体 石药欧意红霉素肠溶片
创建实体 天方药业盐酸克林霉素胶囊
创建实体 唐山红星硝酸益康唑溶液
创建实体 江西诚志永丰蛇胆川贝液
创建实体 联邦左福康盐酸左氧氟沙星胶
创建实体 东药替硝唑片
创建实体 天丁噻托溴铵粉雾剂
创建实体 埃斯玛隆
创建实体 二天堂联苯苄唑乳膏
创建实体 振兴氨茶碱片
创建实体 申优
创建实体 亨瑞达制药甲硝唑片
创建实体 斯皮仁诺
创建实体 健生制药阿奇霉素分散片
创建实体 大连天宇制药阿奇霉素胶囊
创建实体 银诺克肺宝蛤蚧定喘丸
创建实体 大连药业托西酸舒

### 创建 症状 实体

In [37]:
for each in symptoms:
    node = Node('Symptom', name=each)
    g.create(node)
    print('创建实体 {}'.format(each))

创建实体 心源性呼吸窘迫
创建实体 闫铁
创建实体 胸痛
创建实体 头晕
创建实体 肺泡灌洗液可见...
创建实体 呕吐
创建实体 口唇青紫
创建实体 恶心
创建实体 纤毛上皮细胞损伤脱落
创建实体 抽搐
创建实体 咯血伴发热
创建实体 紧张性头晕
创建实体 痉挛性咳嗽
创建实体 痰呈粘液脓性
创建实体 发烧
创建实体 肺纤维化
创建实体 昏迷
创建实体 眼睛痒
创建实体 谵妄
创建实体 目赤
创建实体 紫绀
创建实体 喘息
创建实体 急性面容
创建实体 纵隔浊音界扩大
创建实体 腹泻
创建实体 盗汗
创建实体 黏稠或脓性痰伴...
创建实体 体重减轻
创建实体 惊厥
创建实体 气急
创建实体 肺部出血
创建实体 横膈低平
创建实体 蛋白尿
创建实体 咽部异物感
创建实体 间歇性头痛
创建实体 肺部啰音
创建实体 骨膜炎
创建实体 乏力
创建实体 紧张性头痛
创建实体 嗜酸性粒细胞增多
创建实体 肺阴虚
创建实体 吸气时有蝉鸣音
创建实体 出生后即有持续青紫
创建实体 间歇性头晕
创建实体 肺泡炎症
创建实体 咳铁锈色痰
创建实体 鼻塞
创建实体 变应性咳嗽
创建实体 肺纹理增粗
创建实体 面色青紫
创建实体 低热
创建实体 哮鸣音
创建实体 桶状胸
创建实体 闫鹏辉
创建实体 痰中带血丝
创建实体 咳嗽伴胸痛
创建实体 冬春季的慢性咳...
创建实体 多发性神经炎
创建实体 呼吸困难
创建实体 小支气管粘膜水肿
创建实体 气短
创建实体 咳出棕色痰栓
创建实体 腱反射消失
创建实体 感觉障碍
创建实体 湿啰音
创建实体 毓卓
创建实体 胸闷憋气
创建实体 咯血
创建实体 发热伴咳嗽、咯...
创建实体 化学性支气管炎
创建实体 咳嗽伴哮鸣音
创建实体 呼气音延长
创建实体 胸闷
创建实体 口唇和甲床略带青紫
创建实体 浅感觉减退或缺失
创建实体 畏寒
创建实体 呼吸音减弱
创建实体 耸肩喘息


## 创建知识图谱关系（连接、边）

### 样例代码

In [38]:
# # Neo4j样例代码

# # 删除所有实体和关系
# cypher = 'MATCH (n) DETACH DELETE n'
# g.run(cypher)

In [39]:
# # 创建关系 样例代码
# start_node = 'Disease'
# end_node = 'Check'
# p = '百日咳'
# q = '血常规'
# rel_type = 'need_check'
# rel_name = '诊断检查'

# # Cypher语句
# query = "match(p:%s),(q:%s) where p.name='%s' and q.name='%s' create (p)-[rel:%s{name:'%s'}]->(q)" % (start_node, end_node, p, q, rel_type, rel_name)
# print(query)
# g.run(query) # 运行 Cypher 语句


In [40]:
def create_relationship(start_node, end_node, edges, rel_type, rel_name):
    '''创建关系函数'''
    for edge in edges:
        p = edge[0]
        q = edge[1]
        # 创建关系的 Cypher 语句
        query = "match(p:%s),(q:%s) where p.name='%s' and q.name='%s' create (p)-[rel:%s{name:'%s'}]->(q)" % (start_node, end_node, p, q, rel_type, rel_name)
        try:
            g.run(query) # 运行 Cypher 语句
            print('创建关系 {}-{}->{}'.format(p, rel_type, q))
        except Exception as e:
            print(e)

### 创建所有关系

In [41]:
create_relationship('Disease', 'Food', rels_recommandeat, 'recommand_eat', '推荐食谱')
create_relationship('Disease', 'Food', rels_noteat, 'no_eat', '忌吃')
create_relationship('Disease', 'Food', rels_doeat, 'do_eat', '宜吃')
create_relationship('Department', 'Department', rels_department, 'belongs_to', '属于')
create_relationship('Disease', 'Drug', rels_commonddrug, 'common_drug', '常用药品')
create_relationship('Producer', 'Drug', rels_drug_producer, 'drugs_of', '生产药品')
create_relationship('Disease', 'Drug', rels_recommanddrug, 'recommand_drug', '好评药品')
create_relationship('Disease', 'Check', rels_check, 'need_check', '诊断检查')
create_relationship('Disease', 'Symptom', rels_symptom, 'has_symptom', '症状')
create_relationship('Disease', 'Disease', rels_acompany, 'acompany_with', '并发症')
create_relationship('Disease', 'Department', rels_category, 'belongs_to', '所属科室')

创建关系 百日咳-recommand_eat->清蒸鸡蛋羹
创建关系 百日咳-recommand_eat->百合双耳鸡蛋羹
创建关系 百日咳-recommand_eat->排骨汤
创建关系 百日咳-recommand_eat->罗汉果雪耳鸡汤
创建关系 百日咳-recommand_eat->小黄瓜凉拌面
创建关系 百日咳-recommand_eat->黄瓜三丝汤
创建关系 百日咳-recommand_eat->黄瓜拌兔丝
创建关系 百日咳-recommand_eat->黄瓜拌皮丝
创建关系 苯中毒-recommand_eat->豆腐干炒韭菜
创建关系 苯中毒-recommand_eat->素炒小白菜
创建关系 苯中毒-recommand_eat->白菜蛋花粥
创建关系 喘息样支气管炎-recommand_eat->紫苏粥
创建关系 喘息样支气管炎-recommand_eat->菊花芦根茶
创建关系 喘息样支气管炎-recommand_eat->生芦根粥
创建关系 喘息样支气管炎-recommand_eat->鲜芦根粥
创建关系 喘息样支气管炎-recommand_eat->薏仁党参粥
创建关系 喘息样支气管炎-recommand_eat->党参粥
创建关系 喘息样支气管炎-recommand_eat->黄芪粥
创建关系 喘息样支气管炎-recommand_eat->鸡蛋木耳粥
创建关系 成人呼吸窘迫综合征-recommand_eat->百合糖粥
创建关系 成人呼吸窘迫综合征-recommand_eat->百合粥
创建关系 成人呼吸窘迫综合征-recommand_eat->百合花生汤
创建关系 成人呼吸窘迫综合征-recommand_eat->首乌百合粥
创建关系 成人呼吸窘迫综合征-recommand_eat->薏米莲子粥
创建关系 成人呼吸窘迫综合征-recommand_eat->牛奶玉米汤
创建关系 成人呼吸窘迫综合征-recommand_eat->萝卜豆腐汤
创建关系 成人呼吸窘迫综合征-recommand_eat->瓜粒杂锦汤
创建关系 大叶性肺炎-recommand_eat->奶汤锅子鱼
创建关系 大叶性肺炎-recommand_eat->酱豆腐汁烧猪肉
创建关系 大叶性肺炎-recommand_eat->百合汤
创建关系 大叶性肺炎-recommand_

创建关系 华南药业红霉素肠溶片-drugs_of->红霉素肠溶片
创建关系 佐今明百咳静糖浆-drugs_of->百咳静糖浆
创建关系 恒益药业琥乙红霉素颗粒-drugs_of->琥乙红霉素颗粒
创建关系 利君沙-drugs_of->琥乙红霉素颗粒
创建关系 桂林南药布美他尼片-drugs_of->布美他尼片
创建关系 雄巴拉曲神水十一味金色丸-drugs_of->十一味金色丸
创建关系 西藏甘露仁青芒觉-drugs_of->仁青芒觉
创建关系 金诃藏药大月晶丸-drugs_of->大月晶丸
创建关系 金诃藏药仁青芒觉-drugs_of->仁青芒觉
创建关系 苏肽生-drugs_of->注射用鼠神经生长因子
创建关系 新峰药业盐酸利多卡因注射液-drugs_of->盐酸利多卡因注射液
创建关系 海斯制药盐酸利多卡因注射液-drugs_of->盐酸利多卡因注射液
创建关系 西藏甘露十一味金色丸-drugs_of->十一味金色丸
创建关系 卓峰地塞米松磷酸钠注射液-drugs_of->地塞米松磷酸钠注射液
创建关系 白云山天心地塞米松磷酸钠注-drugs_of->地塞米松磷酸钠注射液
创建关系 涟水制药地塞米松磷酸钠注射-drugs_of->地塞米松磷酸钠注射液
创建关系 金耀药业地塞米松磷酸钠注射-drugs_of->地塞米松磷酸钠注射液
创建关系 遂成药业地塞米松磷酸钠注射-drugs_of->地塞米松磷酸钠注射液
创建关系 恩经复-drugs_of->注射用鼠神经生长因子
创建关系 金路捷-drugs_of->注射用鼠神经生长因子
创建关系 新亚注射用硫代硫酸钠-drugs_of->注射用硫代硫酸钠
创建关系 北京永康药业盐酸利多卡因注-drugs_of->盐酸利多卡因注射液
创建关系 北京益民药业盐酸利多卡因注-drugs_of->盐酸利多卡因注射液
创建关系 中国大冢盐酸利多卡因注射液-drugs_of->盐酸利多卡因注射液
创建关系 盐酸利多卡因注射液-drugs_of->盐酸利多卡因注射液
创建关系 全星制药注射用布美他尼-drugs_of->注射用布美他尼
创建关系 皇隆制药注射用呋塞米-drugs_of->注射用呋塞米
创建关系 信谊布地奈德气雾剂-drugs_of->布地奈德气雾剂
创建关系 桑海制药喷托维林氯化铵糖浆-drugs_of->喷托维林氯

创建关系 国医堂制药如意定喘丸-drugs_of->如意定喘丸
创建关系 博利康尼-drugs_of->硫酸特布他林雾化液
创建关系 石药欧意氨茶碱片-drugs_of->氨茶碱片
创建关系 山西云鹏制药氨茶碱片-drugs_of->氨茶碱片
创建关系 广东环球制药茶碱缓释片-drugs_of->茶碱缓释片
创建关系 白敬宇制药氨茶碱片-drugs_of->氨茶碱片
创建关系 湖南九典制药盐酸班布特罗胶-drugs_of->盐酸班布特罗胶囊
创建关系 康裕头孢氨苄胶囊-drugs_of->头孢氨苄胶囊
创建关系 联邦左福康盐酸左氧氟沙星胶-drugs_of->盐酸左氧氟沙星胶囊
创建关系 阿莫仙阿莫西林颗粒-drugs_of->阿莫西林颗粒
创建关系 联邦安必仙氨苄西林胶囊-drugs_of->氨苄西林胶囊
创建关系 江西诚志永丰蛇胆川贝液-drugs_of->蛇胆川贝液
创建关系 鲁抗阿莫西林颗粒-drugs_of->阿莫西林颗粒
创建关系 广东华南依托红霉素片-drugs_of->依托红霉素片
创建关系 孩尔欣头孢克洛颗粒-drugs_of->头孢克洛颗粒
创建关系 银力舒-drugs_of->头孢丙烯分散片
创建关系 达仁堂通宣理肺丸-drugs_of->通宣理肺丸
创建关系 申优-drugs_of->头孢拉定分散片
创建关系 康良头孢拉定胶囊-drugs_of->头孢拉定胶囊
创建关系 安必君氨苄西林胶囊-drugs_of->氨苄西林胶囊
创建关系 佳乐弗乳酸左氧氟沙星片-drugs_of->乳酸左氧氟沙星片
创建关系 银诺克肺宝蛤蚧定喘丸-drugs_of->蛤蚧定喘丸
创建关系 达力芬-drugs_of->头孢克肟颗粒
创建关系 鲁神乙酰螺旋霉素片-drugs_of->乙酰螺旋霉素片
创建关系 鲁抗注射用头孢唑林钠-drugs_of->注射用头孢唑林钠
创建关系 上海新亚头孢克肟分散片-drugs_of->头孢克肟分散片
创建关系 鲁抗麦迪霉素片-drugs_of->麦迪霉素片
创建关系 珠海联邦头孢拉定胶囊-drugs_of->头孢拉定胶囊
创建关系 朗瑞-drugs_of->司帕沙星分散片
创建关系 百日咳-recommand_drug->琥乙红霉素片
创建关系 百日咳-recommand_drug->琥乙红霉素颗粒
创建关系 百日咳-recomma

创建关系 肺气肿-need_check->心电图
创建关系 肺气肿-need_check->无效腔气量／潮气量比值
创建关系 肺气肿-need_check->肺泡气-动脉血氧分压差
创建关系 肺炎杆菌肺炎-need_check->痰液细菌涂片检查
创建关系 肺炎杆菌肺炎-need_check->痰液细菌培养
创建关系 肺炎杆菌肺炎-need_check->血常规
创建关系 肺泡蛋白质沉积症-has_symptom->紫绀
创建关系 肺泡蛋白质沉积症-has_symptom->胸痛
创建关系 肺泡蛋白质沉积症-has_symptom->呼吸困难
创建关系 肺泡蛋白质沉积症-has_symptom->乏力
创建关系 肺泡蛋白质沉积症-has_symptom->毓卓
创建关系 百日咳-has_symptom->吸气时有蝉鸣音
创建关系 百日咳-has_symptom->痉挛性咳嗽
创建关系 百日咳-has_symptom->胸闷
创建关系 百日咳-has_symptom->肺阴虚
创建关系 百日咳-has_symptom->抽搐
创建关系 百日咳-has_symptom->低热
创建关系 百日咳-has_symptom->闫鹏辉
创建关系 百日咳-has_symptom->惊厥
创建关系 苯中毒-has_symptom->恶心
创建关系 苯中毒-has_symptom->抽搐
创建关系 苯中毒-has_symptom->感觉障碍
创建关系 喘息样支气管炎-has_symptom->耸肩喘息
创建关系 喘息样支气管炎-has_symptom->哮鸣音
创建关系 喘息样支气管炎-has_symptom->纤毛上皮细胞损伤脱落
创建关系 喘息样支气管炎-has_symptom->变应性咳嗽
创建关系 喘息样支气管炎-has_symptom->化学性支气管炎
创建关系 喘息样支气管炎-has_symptom->喘息
创建关系 喘息样支气管炎-has_symptom->冬春季的慢性咳...
创建关系 喘息样支气管炎-has_symptom->咳嗽伴哮鸣音
创建关系 成人呼吸窘迫综合征-has_symptom->呼吸困难
创建关系 成人呼吸窘迫综合征-has_symptom->紫绀
创建关系 成人呼吸窘迫综合征-has_symptom->心源性呼吸窘迫
创建关系 大量羊水吸入-has